In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
#build the vocabulary of characters and mappings to/from integes

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
# build the dataset
block_size = 3 # context length : how many characters do we take to predict the next one
def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    return torch.tensor(X), torch.tensor(Y)

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr, Ytr = build_dataset(words[:n1]) # 80 %
Xdev, Ydev = build_dataset(words[n1:n2]) #10%
Xte, Yte = build_dataset(words[n2:]) # 10%
print(Xtr.shape, Ytr.shape)
print(Xdev.shape, Ydev.shape)
print(Xte.shape, Yte.shape)

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
# utility function we will use later when comparing manual gradients to Pytorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt-t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s}| approximate:{str(app):5s}| maxdiff:{maxdiff}')

In [52]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn(vocab_size, n_embd, generator=g)
W1 = torch.randn(n_embd*block_size, n_hidden, generator=g) *(5/3)/((n_embd*block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
W2 = torch.randn(n_hidden, vocab_size, generator=g) *0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1


#batch norm parameters
bngain = torch.randn((1,n_hidden))*0.1 + 1.0
bnbias = torch.randn((1,n_hidden))*0.1

#Note : initializing many of these parameters in non standardways
# because sometimes initializing with e.g all zeros could mask an incorrect
# implementation of the backward pass

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [53]:
batch_size = 32
n = batch_size
# a shorter variable for convenience.
ix = torch.randint(0, Xtr.shape[0], (n, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [54]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn-bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar+1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain*bnraw + bnbias

#Non linearity

h = torch.tanh(hpreact)

#linear layer  2
logits = h @ W2 + b2
#cross entoryp loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits-logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims = True)
counts_sum_inv = counts_sum**-1
probs = counts*counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

#pytorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
         norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv,bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3390, grad_fn=<NegBackward0>)

In [16]:
logprobs.shape

torch.Size([32, 27])

In [21]:
- logprobs[range(n), Yb]

tensor(3.3385, grad_fn=<NegBackward0>)

In [60]:
# loss  =  - (a+b+c)/3

# dloss/da = -1/3

# Now in the above case derivate is only
#  for the numbers which are Yb indexed and all others provide zero gradient

# How can we pluck those out and use it to calculate backpropogation

# One way to achieve this is to just create a tensor same as logprobs

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

dprobs = (1.0/ probs) * dlogprobs 
# This makes the entities which are low proability ,
#  boost since you are taking an inverse, so that learning happens more here.

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim = True)
dcounts = counts_sum_inv * dprobs
dcounts_sum = (-counts_sum**-2)*dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum
dnorm_logits = counts * dcounts
dlogits = dnorm_logits.clone()
dlogit_maxes = (-dnorm_logits).sum(1, keepdim = True)
dlogits+= F.one_hot(logits.max(1).indices, num_classes = logits.shape[1]) * dlogit_maxes
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact  = (1.0 - h**2) * dh

dbngain = (bnraw * dhpreact).sum(0, keepdim = True)
dbnraw =  bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)

dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim = True)

dbnvar  = (-0.5 * (bnvar+1e-5)**-1.5)*dbnvar_inv

dbndiff2 = (1/(n-1))* torch.ones_like(bndiff2)* dbnvar

dbndiff+= 2*bndiff * dbndiff2

dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)

dhprebn+= 1.0/n * (torch.ones_like(hprebn)* dbnmeani)


dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]
        dC[ix]+= demb[k, j]


cmp('logprobs', dlogprobs, logprobs)
cmp("probs",  dprobs, probs)
cmp("counts_sum_inv", dcounts_sum_inv, counts_sum_inv)
cmp("counts_sum", dcounts_sum, counts_sum)
cmp("counts", dcounts, counts)
cmp('norm_logits', dcounts, counts)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnraw', dbnraw, bnraw)
cmp('bnbias', dbnbias, bnbias)
cmp('bndiff', dbndiff, bndiff)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)

cmp('hprebn', dhprebn, hprebn)
cmp('bnmeani', dbnmeani, bnmeani)

cmp('embcat', dembcat, embcat)

cmp('W1', dW1, W1)
cmp('b1', db1, b1)

cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True | approximate:True | maxdiff:0.0
probs           | exact: True | approximate:True | maxdiff:0.0
counts_sum_inv  | exact: True | approximate:True | maxdiff:0.0
counts_sum      | exact: True | approximate:True | maxdiff:0.0
counts          | exact: True | approximate:True | maxdiff:0.0
norm_logits     | exact: True | approximate:True | maxdiff:0.0
logit_maxes     | exact: True | approximate:True | maxdiff:0.0
logits          | exact: True | approximate:True | maxdiff:0.0
h               | exact: True | approximate:True | maxdiff:0.0
W2              | exact: True | approximate:True | maxdiff:0.0
b2              | exact: True | approximate:True | maxdiff:0.0
hpreact         | exact: True | approximate:True | maxdiff:0.0
bngain          | exact: True | approximate:True | maxdiff:0.0
bnraw           | exact: True | approximate:True | maxdiff:0.0
bnbias          | exact: True | approximate:True | maxdiff:0.0
bndiff          | exact: True | approximate:True | maxd

In [25]:
counts.shape, counts_sum_inv.shape, counts_sum.shape

(torch.Size([32, 27]), torch.Size([32, 1]), torch.Size([32, 1]))

In [29]:
counts.sum(1 , keepdim=True).shape

torch.Size([32, 1])

In [38]:
dlogit_maxes

tensor([[ 3.0268e-09],
        [-4.6566e-09],
        [ 0.0000e+00],
        [ 1.8626e-09],
        [ 4.6566e-10],
        [ 2.0955e-09],
        [-5.5879e-09],
        [ 4.6566e-10],
        [-4.6566e-10],
        [-2.0955e-09],
        [-1.8626e-09],
        [ 4.6566e-10],
        [ 2.3283e-10],
        [-4.6566e-09],
        [ 3.4925e-09],
        [ 2.5611e-09],
        [-3.7253e-09],
        [ 1.6298e-09],
        [ 1.3970e-09],
        [-6.2864e-09],
        [ 0.0000e+00],
        [-6.9849e-10],
        [ 3.2596e-09],
        [-2.7940e-09],
        [ 0.0000e+00],
        [ 3.2596e-09],
        [ 6.9849e-10],
        [ 1.6298e-09],
        [-9.3132e-10],
        [ 3.2596e-09],
        [ 5.5879e-09],
        [ 2.3283e-09]], grad_fn=<SumBackward1>)

In [44]:
hpreact.shape,bngain.shape,bnraw.shape, bnbias.shape

(torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]))

In [62]:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.339038610458374 diff: 2.384185791015625e-07


In [63]:
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb]-=1
dlogits/=n

cmp('logits', dlogits, logits)

logits          | exact: False| approximate:True | maxdiff:4.423782229423523e-09
